In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import sys
import os

root = "/project/wyin/jlee/ml-project/dos-prediction"
util_loc = os.path.join(root, "utils")
data_loc = os.path.join(root, "data")
sys.path.append(util_loc)
#from utilities import LitNeuralNet, DosDataModule

In [4]:
architectures = [[]]
for n_layers in range(1, 9):
    #Uniform architecture
    n_neurons = 64
    while n_neurons <= 1024:
        layer_sizes = [n_neurons] * n_layers
        n_neurons *= 2
        #print(layer_sizes)
        architectures.append(layer_sizes)
    
    #Up to 1024, linearly
    if n_layers >= 2:
        end_neurons = 64
        while end_neurons <= 1024:
            layer_sizes = []
            for i in range(1, n_layers + 1):
                layer_sizes.append(i * (end_neurons // n_layers))

            end_neurons *= 2
            architectures.append(layer_sizes)

for layer_sizes in architectures:
    layer_sizes.insert(0, 3)
    layer_sizes.append(193)
    print(layer_sizes)
    
print(len(architectures))

[3, 193]
[3, 64, 193]
[3, 128, 193]
[3, 256, 193]
[3, 512, 193]
[3, 1024, 193]
[3, 64, 64, 193]
[3, 128, 128, 193]
[3, 256, 256, 193]
[3, 512, 512, 193]
[3, 1024, 1024, 193]
[3, 32, 64, 193]
[3, 64, 128, 193]
[3, 128, 256, 193]
[3, 256, 512, 193]
[3, 512, 1024, 193]
[3, 64, 64, 64, 193]
[3, 128, 128, 128, 193]
[3, 256, 256, 256, 193]
[3, 512, 512, 512, 193]
[3, 1024, 1024, 1024, 193]
[3, 21, 42, 63, 193]
[3, 42, 84, 126, 193]
[3, 85, 170, 255, 193]
[3, 170, 340, 510, 193]
[3, 341, 682, 1023, 193]
[3, 64, 64, 64, 64, 193]
[3, 128, 128, 128, 128, 193]
[3, 256, 256, 256, 256, 193]
[3, 512, 512, 512, 512, 193]
[3, 1024, 1024, 1024, 1024, 193]
[3, 16, 32, 48, 64, 193]
[3, 32, 64, 96, 128, 193]
[3, 64, 128, 192, 256, 193]
[3, 128, 256, 384, 512, 193]
[3, 256, 512, 768, 1024, 193]
[3, 64, 64, 64, 64, 64, 193]
[3, 128, 128, 128, 128, 128, 193]
[3, 256, 256, 256, 256, 256, 193]
[3, 512, 512, 512, 512, 512, 193]
[3, 1024, 1024, 1024, 1024, 1024, 193]
[3, 12, 24, 36, 48, 60, 193]
[3, 25, 50, 75, 

In [10]:
architectures = []
for n_layers in [4, 6]:
    
    #Up to 1024, linearly
    end_neurons = 256
    while end_neurons <= 1024:
        layer_sizes = []
        for i in range(1, n_layers + 1):
            layer_sizes.append(i * (end_neurons // n_layers))
        
        end_neurons *= 2
        architectures.append(layer_sizes)
    
    #Up to 1024, exponentially
    end_neurons = 256
    while end_neurons <= 1024:
        layer_sizes = [end_neurons]
        for i in range(n_layers - 1):
            layer_sizes.insert(0, layer_sizes[0] // 2)
            
        end_neurons *= 2
        
        architectures.append(layer_sizes)
    
for layer_sizes in architectures:
    layer_sizes.insert(0, 3)
    layer_sizes.append(301)
    print(f"{layer_sizes},")
    

[3, 64, 128, 192, 256, 301],
[3, 128, 256, 384, 512, 301],
[3, 256, 512, 768, 1024, 301],
[3, 32, 64, 128, 256, 301],
[3, 64, 128, 256, 512, 301],
[3, 128, 256, 512, 1024, 301],
[3, 42, 84, 126, 168, 210, 252, 301],
[3, 85, 170, 255, 340, 425, 510, 301],
[3, 170, 340, 510, 680, 850, 1020, 301],
[3, 8, 16, 32, 64, 128, 256, 301],
[3, 16, 32, 64, 128, 256, 512, 301],
[3, 32, 64, 128, 256, 512, 1024, 301],


In [14]:
architectures = []
for n_layers in range(21):
    increment = round(298 / (n_layers + 1))
    layer_sizes = [3 + i * (increment) for i in range(n_layers + 1)]
    layer_sizes.append(301)
    architectures.append(layer_sizes)

increment_med = (512 - 3) // 6
layer_sizes_med = [3 + i * (increment_med) for i in range(7)]
layer_sizes_med.append(301)

increment_large = (1024 - 3) // 6
layer_sizes_large = [3 + i * (increment_large) for i in range(7)]
layer_sizes_large.append(301)

architectures.extend([layer_sizes_med, layer_sizes_large])

[3, 301]
[3, 152, 301]
[3, 102, 201, 301]
[3, 77, 151, 225, 301]
[3, 63, 123, 183, 243, 301]
[3, 53, 103, 153, 203, 253, 301]
[3, 46, 89, 132, 175, 218, 261, 301]
[3, 40, 77, 114, 151, 188, 225, 262, 301]
[3, 36, 69, 102, 135, 168, 201, 234, 267, 301]
[3, 33, 63, 93, 123, 153, 183, 213, 243, 273, 301]
[3, 30, 57, 84, 111, 138, 165, 192, 219, 246, 273, 301]
[3, 28, 53, 78, 103, 128, 153, 178, 203, 228, 253, 278, 301]
[3, 26, 49, 72, 95, 118, 141, 164, 187, 210, 233, 256, 279, 301]
[3, 24, 45, 66, 87, 108, 129, 150, 171, 192, 213, 234, 255, 276, 301]
[3, 23, 43, 63, 83, 103, 123, 143, 163, 183, 203, 223, 243, 263, 283, 301]
[3, 22, 41, 60, 79, 98, 117, 136, 155, 174, 193, 212, 231, 250, 269, 288, 301]
[3, 21, 39, 57, 75, 93, 111, 129, 147, 165, 183, 201, 219, 237, 255, 273, 291, 301]
[3, 20, 37, 54, 71, 88, 105, 122, 139, 156, 173, 190, 207, 224, 241, 258, 275, 292, 301]
[3, 19, 35, 51, 67, 83, 99, 115, 131, 147, 163, 179, 195, 211, 227, 243, 259, 275, 291, 301]
[3, 18, 33, 48, 63, 78, 9

In [6]:
batch_size = 1000
learning_rate = 0.001
dos_data = DosDataModule(data_loc, batch_size)

for layer_sizes in architectures:
    print(f"testing layer sizes: {layer_sizes}")
    logger = TensorBoardLogger('logs/architecture-search', name = f'{layer_sizes}')
    trainer = pl.Trainer(enable_checkpointing=False, max_time="00:00:02:00", logger = logger)

    model = LitNeuralNet(layer_sizes, lr = learning_rate)
    trainer.fit(model, datamodule=dos_data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/architecture-search/[3, 64, 64, 301]


testing layer sizes: [3, 64, 64, 301]



  | Name         | Type       | Params
--------------------------------------------
0 | forward_prop | Sequential | 24.0 K
--------------------------------------------
24.0 K    Trainable params
0         Non-trainable params
24.0 K    Total params
0.096     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/wyin/.conda/envs/ML-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/wyin/.conda/envs/ML-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Time limit reached. Elapsed time is 0:02:00. Signaling Trainer to stop.


Validation: 0it [00:00, ?it/s]